<a href="https://colab.research.google.com/github/d-kleine/Advent_of_HayStack/blob/main/07_Arize_Phoenix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advent of Haystack - Day 7
_Make a copy of this Colab to start_

Santa collapsed in his chair in a huff. "What's wrong?" asked Mrs Claus.

"There's just too many toys to check and not enough time! Christmas is almost here!"

"Well can't you just check some of them?"

"I wish it were that easy! But my elves make so many different toys, and we have to make sure every kid gets the right one!"

Elf Jane couldn't help overhearing from the next room. She was a regular attendee at the local north pole hackathon, and thought she might have a solution. She'd learned a lot about evaluation recently, and thought she could build an LLM Judge to help.

**For this challenge, you need to help Elf Jane and complete the code cells with `#TODO` text**

 <img src='https://github.com/Jgilhuly/phoenix-assets/blob/main/images/socal/advent-of-haystack-1.jpeg?raw=true' width=500px>

## Installation

In [1]:
# !pip install -q arize-phoenix==6.1.0 haystack-ai==2.7.0 openinference-instrumentation-haystack==0.1.13 httpx<0.28

## Data

Elf Jane started by checking out the big elf database of christmas wishlists (aka the BEDCW).

In [2]:
children = [
    {'name': 'Timmy', 'age': 7, 'likes': 'Lego', 'dislikes': 'Vegetables', 'list': 'nice'},
    {'name': 'Tommy', 'age': 9, 'likes': 'Sports Equipment', 'dislikes': 'Reading', 'list': 'naughty'},
    {'name': 'Tammy', 'age': 8, 'likes': 'Art Supplies', 'dislikes': 'Loud Noises', 'list': 'nice'},
    {'name': 'Tina', 'age': 6, 'likes': 'Science Kits', 'dislikes': 'Spicy Food', 'list': 'nice'},
    {'name': 'Toby', 'age': 10, 'likes': 'Video Games', 'dislikes': 'Early Mornings', 'list': 'nice'},
    {'name': 'Tod', 'age': 5, 'likes': 'Musical Instruments', 'dislikes': 'Bath Time', 'list': 'nice'},
    {'name': 'Todd', 'age': 8, 'likes': 'Remote Control Cars', 'dislikes': 'Homework', 'list': 'naughty'},
    {'name': 'Tara', 'age': 7, 'likes': 'Magic Sets', 'dislikes': 'Thunder', 'list': 'nice'},
    {'name': 'Teri', 'age': 9, 'likes': 'Building Blocks', 'dislikes': 'Broccoli', 'list': 'nice'},
    {'name': 'Trey', 'age': 6, 'likes': 'Board Games', 'dislikes': 'Bedtime', 'list': 'nice'},
    {'name': 'Tyler', 'age': 8, 'likes': 'Action Figures', 'dislikes': 'Cleaning', 'list': 'nice'},
    {'name': 'Tracy', 'age': 7, 'likes': 'Dolls', 'dislikes': 'Dark', 'list': 'nice'},
    {'name': 'Tony', 'age': 9, 'likes': 'Chemistry Sets', 'dislikes': 'Dentist', 'list': 'nice'},
    {'name': 'Theo', 'age': 6, 'likes': 'Puzzles', 'dislikes': 'Shots', 'list': 'nice'},
    {'name': 'Terry', 'age': 10, 'likes': 'Model Trains', 'dislikes': 'Chores', 'list': 'naughty'},
    {'name': 'Tessa', 'age': 5, 'likes': 'Stuffed Animals', 'dislikes': 'Time Out', 'list': 'nice'},
    {'name': 'Troy', 'age': 8, 'likes': 'Robots', 'dislikes': 'Naps', 'list': 'nice'},
    {'name': 'Talia', 'age': 7, 'likes': 'Craft Kits', 'dislikes': 'Spinach', 'list': 'nice'},
    {'name': 'Tyson', 'age': 9, 'likes': 'Microscopes', 'dislikes': 'Cold', 'list': 'nice'},
    {'name': 'Tatum', 'age': 6, 'likes': 'Drawing Sets', 'dislikes': 'Shots', 'list': 'nice'},
]

In [3]:
len(children)

20

# 1. Adding Tracing 📝

Elf Jane knew that the elves were busy, and didn't always log their toy making process. She knew that she'd first need to trace the toy making process using Arize Phoenix.

In [4]:
from getpass import getpass

from phoenix.otel import register
from openinference.instrumentation.haystack import HaystackInstrumentor

# TODO: Add Phoenix tracing with Haystack: https://docs.arize.com/phoenix/tracing/integrations-tracing/haystack
# There are many ways to launch Phoenix - the simplest way for this example is to use the "Notebook" option

import os
from getpass import getpass

# Add Phoenix API Key for tracing
PHOENIX_API_KEY = getpass("PHOENIX_API_KEY")
os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"

# configure the Phoenix tracer
tracer_provider = register(
  project_name="toy_finder", # Default is 'default'
  endpoint="https://app.phoenix.arize.com/v1/traces",
  set_global_tracer_provider=False
)

HaystackInstrumentor().instrument(tracer_provider=tracer_provider)

OpenTelemetry Tracing Details
|  Phoenix Project: toy_finder
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/v1/traces
|  Transport: HTTP
|  Transport Headers: {'api_key': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.



# 2. Trace Toy Making Process 🚂

With tracing in place, Elf Jane had some of her closest elf friends build a batch of toys she could trace.

⭐️ Feel free to replace `OpenAIChatGenerator` with other [ChatGenerators](https://docs.haystack.deepset.ai/docs/generators) supported in Haystack

In [5]:
import json

with open("config.json", "r") as config_file:
    os.environ["OPENAI_API_KEY"] = json.load(config_file)

In [6]:
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.builders import ChatPromptBuilder
from haystack import Pipeline

messages = [
    ChatMessage.from_system("You are a toy maker elf. Your job is to make toys for the nice kids on the nice list. If the child is on the naughty list, give them a 'Rabbit R1'. {{name}} is on the {{list}} list"),
    ChatMessage.from_user("Create a toy for {{name}} that they will like. {{name}} is {{age}} years old and likes {{likes}} and dislikes {{dislikes}}."),
]

builder = ChatPromptBuilder(messages)
chat_generator = OpenAIChatGenerator(model="gpt-4o-mini")

pipeline = Pipeline()
pipeline.add_component("builder", builder)
pipeline.add_component("chat_generator", chat_generator)

pipeline.connect("builder", "chat_generator")

def make_toy(child):
    return pipeline.run({"builder":{**child}})["chat_generator"]["replies"]

In [7]:
for child in children:
    make_toy(child)

# 3. Evaluate Toy Correctness 🔬

Elf Jane was now ready to evaluate the toys she made. She knew that she could use an LLM Judge to evaluate whether the toys matched the child's wishlist. She started by building a judge.

In [8]:
llm_judge_prompt = """
Evaluate the toy for this child, based on their likes and dislikes.

All children on the naughty list get a 'Rabbit R1'. Any other toy given to a naughty child is incorrect.

Respond with a single word: 'correct' or 'incorrect'. Also include a short explanation for your answer.

Description of the child: {description}
Toy: {toy}

*****
Example output:
label: 'correct'
explanation: 'The toy is a Lego set, which is one of the child's likes.'
*****
"""

In [9]:
import phoenix as px

# TODO: Download the traces from Phoenix
# HINT: https://docs.arize.com/phoenix/evaluation/how-to-evals/evaluating-phoenix-traces#download-trace-dataset-from-phoenix

# Download the traces from Phoenix
spans_df = px.Client().get_spans_dataframe(project_name="toy_finder")

# Display the first few rows of the dataframe
spans_df

c:\Users\dk\anaconda3\envs\haystack\Lib\site-packages\phoenix\utilities\client.py:34: UserWarning: The Phoenix server has an unknown version and may have compatibility issues.
  warnings.warn(


,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,context.span_id,context.trace_id,...,attributes.output.mime_type,attributes.input.value,attributes.openinference.span.kind,attributes.input.mime_type,attributes.llm.token_count.prompt,attributes.llm.model_name,attributes.llm.token_count.completion,attributes.llm.input_messages,attributes.llm.token_count.total,attributes.llm.output_messages
context.span_id,,,,,,,,,,,,,,,,,,,,,
7baebf47f46a863e,ChatPromptBuilder (builder),CHAIN,521f234cc07dce51,2024-12-16 21:51:23.666582+00:00,2024-12-16 21:51:23.668118+00:00,OK,,[],7baebf47f46a863e,774a5cde3810f3d1408fc819f2a4dd50,...,application/json,"{""template"": null, ""template_variables"": null,...",CHAIN,application/json,NaN,None,NaN,None,NaN,None
d2baa4ed4f360d34,OpenAIChatGenerator (chat_generator),LLM,521f234cc07dce51,2024-12-16 21:51:24.091367+00:00,2024-12-16 21:51:26.758443+00:00,OK,,[],d2baa4ed4f360d34,774a5cde3810f3d1408fc819f2a4dd50,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,83.0,gpt-4o-mini-2024-07-18,132.0,"[{'message.role': 'system', 'message.content':...",215.0,"[{'message.role': 'assistant', 'message.conten..."
521f234cc07dce51,Pipeline,CHAIN,None,2024-12-16 21:51:23.664021+00:00,2024-12-16 21:51:26.958743+00:00,OK,,[],521f234cc07dce51,774a5cde3810f3d1408fc819f2a4dd50,...,application/json,"{""data"": {""builder"": {""name"": ""Timmy"", ""age"": ...",CHAIN,application/json,NaN,None,NaN,None,NaN,None
0730a6c2215e4590,ChatPromptBuilder (builder),CHAIN,ddc8aa1ed07d444a,2024-12-16 21:51:27.162965+00:00,2024-12-16 21:51:27.165486+00:00,OK,,[],0730a6c2215e4590,77cd537465c9cc7e004dfe013c5710e7,...,application/json,"{""template"": null, ""template_variables"": null,...",CHAIN,application/json,NaN,None,NaN,None,NaN,None
b6db216598a01e66,OpenAIChatGenerator (chat_generator),LLM,ddc8aa1ed07d444a,2024-12-16 21:51:27.367495+00:00,2024-12-16 21:51:33.526215+00:00,OK,,[],b6db216598a01e66,77cd537465c9cc7e004dfe013c5710e7,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,80.0,gpt-4o-mini-2024-07-18,110.0,"[{'message.role': 'system', 'message.content':...",190.0,"[{'message.role': 'assistant', 'message.conten..."
ddc8aa1ed07d444a,Pipeline,CHAIN,None,2024-12-16 21:51:27.162965+00:00,2024-12-16 21:51:33.716082+00:00,OK,,[],ddc8aa1ed07d444a,77cd537465c9cc7e004dfe013c5710e7,...,application/json,"{""data"": {""builder"": {""name"": ""Tommy"", ""age"": ...",CHAIN,application/json,NaN,None,NaN,None,NaN,None
e84af7b26a5350f4,ChatPromptBuilder (builder),CHAIN,7daf51e805681a07,2024-12-16 21:51:33.921735+00:00,2024-12-16 21:51:33.923309+00:00,OK,,[],e84af7b26a5350f4,45a3e45288f61e82f61963102e39fc69,...,application/json,"{""template"": null, ""template_variables"": null,...",CHAIN,application/json,NaN,None,NaN,None,NaN,None
f7a379b24fcdaa8b,OpenAIChatGenerator (chat_generator),LLM,7daf51e805681a07,2024-12-16 21:51:34.125977+00:00,2024-12-16 21:51:37.710418+00:00,OK,,[],f7a379b24fcdaa8b,45a3e45288f61e82f61963102e39fc69,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,82.0,gpt-4o-mini-2024-07-18,179.0,"[{'message.role': 'system', 'message.content':...",261.0,"[{'message.role': 'assistant', 'message.conten..."
7daf51e805681a07,Pipeline,CHAIN,None,2024-12-16 21:51:33.921735+00:00,2024-12-16 21:51:37.915158+00:00,OK,,[],7daf51e805681a07,45a3e45288f61e82f61963102e39fc69,...,application/json,"{""data"": {""builder"": {""name"": ""Tammy"", ""age"": ...",CHAIN,application/json,NaN,None,NaN,None,NaN,None


In [10]:
filtered_df = spans_df[(spans_df['span_kind'] == 'LLM')]
filtered_df

,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,context.span_id,context.trace_id,...,attributes.output.mime_type,attributes.input.value,attributes.openinference.span.kind,attributes.input.mime_type,attributes.llm.token_count.prompt,attributes.llm.model_name,attributes.llm.token_count.completion,attributes.llm.input_messages,attributes.llm.token_count.total,attributes.llm.output_messages
context.span_id,,,,,,,,,,,,,,,,,,,,,
d2baa4ed4f360d34,OpenAIChatGenerator (chat_generator),LLM,521f234cc07dce51,2024-12-16 21:51:24.091367+00:00,2024-12-16 21:51:26.758443+00:00,OK,,[],d2baa4ed4f360d34,774a5cde3810f3d1408fc819f2a4dd50,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,83.0,gpt-4o-mini-2024-07-18,132.0,"[{'message.role': 'system', 'message.content':...",215.0,"[{'message.role': 'assistant', 'message.conten..."
b6db216598a01e66,OpenAIChatGenerator (chat_generator),LLM,ddc8aa1ed07d444a,2024-12-16 21:51:27.367495+00:00,2024-12-16 21:51:33.526215+00:00,OK,,[],b6db216598a01e66,77cd537465c9cc7e004dfe013c5710e7,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,80.0,gpt-4o-mini-2024-07-18,110.0,"[{'message.role': 'system', 'message.content':...",190.0,"[{'message.role': 'assistant', 'message.conten..."
f7a379b24fcdaa8b,OpenAIChatGenerator (chat_generator),LLM,7daf51e805681a07,2024-12-16 21:51:34.125977+00:00,2024-12-16 21:51:37.710418+00:00,OK,,[],f7a379b24fcdaa8b,45a3e45288f61e82f61963102e39fc69,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,82.0,gpt-4o-mini-2024-07-18,179.0,"[{'message.role': 'system', 'message.content':...",261.0,"[{'message.role': 'assistant', 'message.conten..."
bb7e5cb59ae5db95,OpenAIChatGenerator (chat_generator),LLM,833228d882b6d313,2024-12-16 21:51:38.324694+00:00,2024-12-16 21:51:42.245950+00:00,OK,,[],bb7e5cb59ae5db95,1487f2d78895291d483db0bda1b08622,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,82.0,gpt-4o-mini-2024-07-18,195.0,"[{'message.role': 'system', 'message.content':...",277.0,"[{'message.role': 'assistant', 'message.conten..."
026b77738d9bed0c,OpenAIChatGenerator (chat_generator),LLM,aa85d64112a0d672,2024-12-16 21:51:42.766519+00:00,2024-12-16 21:51:46.619802+00:00,OK,,[],026b77738d9bed0c,e2c322247fa4b87ef8ad24f88354072a,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,83.0,gpt-4o-mini-2024-07-18,220.0,"[{'message.role': 'system', 'message.content':...",303.0,"[{'message.role': 'assistant', 'message.conten..."
40f0c638dadd4888,OpenAIChatGenerator (chat_generator),LLM,ee4418de4fc0f956,2024-12-16 21:51:47.233549+00:00,2024-12-16 21:51:51.535303+00:00,OK,,[],40f0c638dadd4888,14ef4d2028250236596a2b3504f8814b,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,81.0,gpt-4o-mini-2024-07-18,213.0,"[{'message.role': 'system', 'message.content':...",294.0,"[{'message.role': 'assistant', 'message.conten..."
82aec0b1fee9960b,OpenAIChatGenerator (chat_generator),LLM,db635a88a2357f9b,2024-12-16 21:51:52.148445+00:00,2024-12-16 21:51:56.452746+00:00,OK,,[],82aec0b1fee9960b,ed643778ba501492e3b96bace8abbb18,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,81.0,gpt-4o-mini-2024-07-18,215.0,"[{'message.role': 'system', 'message.content':...",296.0,"[{'message.role': 'assistant', 'message.conten..."
41badd20c9be021a,OpenAIChatGenerator (chat_generator),LLM,eaf20ff11c341070,2024-12-16 21:51:57.002275+00:00,2024-12-16 21:52:00.238440+00:00,OK,,[],41badd20c9be021a,39d8a597118c3ec9845cccd9727eb26b,...,application/json,"{""messages"": [""ChatMessage(content=\""You are a...",LLM,application/json,80.0,gpt-4o-mini-2024-07-18,197.0,"[{'message.role': 'system', 'message.content':...",277.0,"[{'message.role': 'assistant', 'message.conten..."
2af17444af0494a3,Open

In [11]:
input_messages = filtered_df['attributes.llm.input_messages']
output_messages = filtered_df['attributes.llm.output_messages']

# Function to extract 'message.content'
def extract_message_content(messages):
    return [message['message.content'] for message in messages]

# Extract content from input and output messages
input_contents = input_messages.apply(extract_message_content)
output_contents = output_messages.apply(extract_message_content)

In [12]:
input_contents.iloc[0]

["You are a toy maker elf. Your job is to make toys for the nice kids on the nice list. If the child is on the naughty list, give them a 'Rabbit R1'. Timmy is on the nice list",
 'Create a toy for Timmy that they will like. Timmy is 7 years old and likes Lego and dislikes Vegetables.']

In [13]:
output_contents.iloc[0]

['For Timmy, I would create a "Lego Adventure Set"! This set would include a variety of themed Lego kits such as a pirate ship, a jungle explorer scene, and a space rocket. Each set would have unique mini-figures, accessories, and interactive elements, like a working compass on the explorer set or a treasure chest on the pirate ship. \n\nTo make it even more special, we could include a special "Timmy\'s Adventure" instruction booklet with fun challenges and stories to inspire his imagination. \n\nThis Lego Adventure Set will keep Timmy engaged while nurturing his creativity, and of course, it has absolutely nothing to do with vegetables!']

In [14]:
# Extract the last sentence from each input
extracted_descriptions = input_contents.apply(lambda x: ' '.join(x))
print(extracted_descriptions)

context.span_id
d2baa4ed4f360d34    You are a toy maker elf. Your job is to make t...
b6db216598a01e66    You are a toy maker elf. Your job is to make t...
f7a379b24fcdaa8b    You are a toy maker elf. Your job is to make t...
bb7e5cb59ae5db95    You are a toy maker elf. Your job is to make t...
026b77738d9bed0c    You are a toy maker elf. Your job is to make t...
40f0c638dadd4888    You are a toy maker elf. Your job is to make t...
82aec0b1fee9960b    You are a toy maker elf. Your job is to make t...
41badd20c9be021a    You are a toy maker elf. Your job is to make t...
2af17444af0494a3    You are a toy maker elf. Your job is to make t...
d9bae836f0e996cf    You are a toy maker elf. Your job is to make t...
6a8a3e65b0f54e37    You are a toy maker elf. Your job is to make t...
ce614803d1bef3e7    You are a toy maker elf. Your job is to make t...
ced315a4c365616e    You are a toy maker elf. Your job is to make t...
961400bfeab8d4c0    You are a toy maker elf. Your job is to make t...
69e3

In [15]:
extracted_toys = output_contents.apply(lambda x: ' '.join(x))
print(extracted_toys)

context.span_id
d2baa4ed4f360d34    For Timmy, I would create a "Lego Adventure Se...
b6db216598a01e66    Since Tommy is on the naughty list, I’ll be gi...
f7a379b24fcdaa8b    For Tammy, I would create a **Deluxe Art Suppl...
bb7e5cb59ae5db95    For Tina, I would create a fantastic "Science ...
026b77738d9bed0c    For Toby, who loves video games, I would creat...
40f0c638dadd4888    For Tod, I would create a colorful **Musical I...
82aec0b1fee9960b    Since Todd is on the naughty list, he will rec...
41badd20c9be021a    For Tara, how about a **Magic Wonder Set**? Th...
2af17444af0494a3    For Teri, I would create a set of **Magical Fa...
d9bae836f0e996cf    For Trey, I’ll create a fun and engaging board...
6a8a3e65b0f54e37    For Tyler, I’ll create an exciting action figu...
ce614803d1bef3e7    For Tracy, I would create a beautiful 'Magic L...
ced315a4c365616e    For Tony, I would create an exciting and engag...
961400bfeab8d4c0    For Theo, I would create a colorful, interacti...
69e3

In [16]:
import pandas as pd

prompts_df = pd.DataFrame({'description': extracted_descriptions, 'toy': extracted_toys})

# Display the new DataFrame
prompts_df.head()

,description,toy
context.span_id,,
d2baa4ed4f360d34,You are a toy maker elf. Your job is to make t...,"For Timmy, I would create a ""Lego Adventure Se..."
b6db216598a01e66,You are a toy maker elf. Your job is to make t...,"Since Tommy is on the naughty list, I’ll be gi..."
f7a379b24fcdaa8b,You are a toy maker elf. Your job is to make t...,"For Tammy, I would create a **Deluxe Art Suppl..."
bb7e5cb59ae5db95,You are a toy maker elf. Your job is to make t...,"For Tina, I would create a fantastic ""Science ..."
026b77738d9bed0c,You are a toy maker elf. Your job is to make t...,"For Toby, who loves video games, I would creat..."


In [17]:
from phoenix.evals import (
    llm_classify,
    OpenAIModel # can swap for another model supported by Phoenix or run open-source models through LiteLLM and Ollama: https://docs.arize.com/phoenix/evaluation/evaluation-models
)

import nest_asyncio
nest_asyncio.apply()

# TODO: Evaluate the traces with the LLM Judge
# HINT: https://docs.arize.com/phoenix/evaluation/how-to-evals/bring-your-own-evaluator#categorical-llm_classify

rails = ["correct", "incorrect"]
eval_results = llm_classify(
    dataframe=prompts_df,
    template=llm_judge_prompt,
    model=OpenAIModel(model="gpt-4o-mini"),
    rails=rails
)

llm_classify |          | 0/20 (0.0%) | ⏳ 00:00<? | ?it/s

In [18]:
eval_results["score"] = eval_results["label"].apply(lambda x: 1 if x == "correct" else 0)
eval_results

,label,exceptions,execution_status,execution_seconds,score
context.span_id,,,,,
d2baa4ed4f360d34,correct,[],COMPLETED,0.696466,1
b6db216598a01e66,correct,[],COMPLETED,1.103583,1
f7a379b24fcdaa8b,correct,[],COMPLETED,0.714420,1
bb7e5cb59ae5db95,correct,[],COMPLETED,0.793503,1
026b77738d9bed0c,correct,[],COMPLETED,0.893053,1
40f0c638dadd4888,correct,[],COMPLETED,0.801042,1
82aec0b1fee9960b,correct,[],COMPLETED,1.209394,1
41badd20c9be021a,correct,[],COMPLETED,1.634045,1
2af17444af0494a3,correct,[],COMPLETED,1.506397,1


In [19]:
from phoenix.trace import SpanEvaluations

# TODO: Upload results into Phoenix
# HINT: https://docs.arize.com/phoenix/evaluation/how-to-evals/evaluating-phoenix-traces#download-trace-dataset-from-phoenix

eval_results["score"] = eval_results["score"].astype(int)
eval_results["label"] = eval_results["label"].astype(str)

px.Client().log_evaluations(SpanEvaluations(eval_name="toy", dataframe=eval_results))

c:\Users\dk\anaconda3\envs\haystack\Lib\site-packages\phoenix\utilities\client.py:34: UserWarning: The Phoenix server has an unknown version and may have compatibility issues.
  warnings.warn(


# 4. View the results in the Arize Phoenix UI 🐦‍🔥

And just like that, Elf Jane had saved Santa hours of time and made sure every kid got the right toy!

In Phoenix, she could see "correct" and "incorrect" labels on all the traces, and even see the explanations for each label!

She couldn't wait to show Santa, and all her friends at the hackathon.

 <img src='https://github.com/Jgilhuly/phoenix-assets/blob/main/images/socal/advent-of-haystack-2.jpeg?raw=true' width=500px>